In [4]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.models import Model, load_model

from glob import glob
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

#import h5py
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image
 
#Importing required libraries.
from tensorflow.keras.models import Sequential
# To initialize neural network
from keras.layers import Convolution2D
# Images are two dimensional, concolution step
from keras.layers import MaxPooling2D
# Pooling step
from keras.layers import Flatten
# Convert pools feature map into this large feature vector
from keras.layers import Dense
#To add fully connected layers
from tensorflow.keras.models import Model, load_model
from keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import Adam 

In [5]:
IMAGE_SIZE =[224,224]
# BATCH_SIZE = 64
train_dir='/content/drive/MyDrive/Final-Dataset4000-SPLIT/train'
val_dir='/content/drive/MyDrive/Final-Dataset4000-SPLIT/val'         #LEARNING_RATE =0.0005

In [6]:
#add preprocessing layer to the front of VGG

vgg=VGG16(input_shape = IMAGE_SIZE +[3],
          weights = 'imagenet',
          include_top= False)
#By keeping include_top as False we have not specified the last layer in the 
# model.Here, we will add our customized last layers.

58900480/58889256 [==============================] - 0s 0us/step


In [7]:
#dont train the existing weights. The weights are already trained
for layer in vgg.layers:
  layer.trainable =False

In [8]:
folders= glob("/content/drive/MyDrive/Final-Dataset4000-SPLIT/train/*")
#To get all the categories under the train folder path in a list we use a glob method.

In [9]:
x=Flatten()(vgg.output) #Removes last laeyrs

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)
#In prediction, while adding a dense layer we have passed the argument
# len(folders), which states the classes normal and COVID. 
#So, len(folders) are two.

model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
model.summary()
#After specifying the last layer, we will now create an object model. 
#Which has standard VGG input, which we have defined above, 
#and prediction as output.

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
#tell the model what cost and optimization method to use
#Now we will compile the defined model object.
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    #optimizer=optimizers.Adam(lr=LEARNING_RATE),
    metrics=['accuracy']
)

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range =4, #randomly rotate my image between 0 and 5 degrees 
                                 width_shift_range=0.1,#shift the image along X-axis by 10% 
                                 height_shift_range=0.1, #shift the image along Y-axis by 10% 
                                 shear_range=0.2, # shear the image by 2% 
                                 zoom_range=0.2, #means zoom-in and zoom-out by 2%
                                 fill_mode='constant',
                                 horizontal_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(224,224),
                                                  batch_size =32,
                                                  class_mode='categorical')
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator= validation_datagen.flow_from_directory(val_dir,
                                                             target_size=(224,224),
                                                             batch_size=32,
                                                             class_mode='categorical')
train_generator.class_indices

Found 5786 images belonging to 2 classes.
Found 2472 images belonging to 2 classes.


{'Covid': 0, 'Non-Covid': 1}

In [14]:
EPOCHS_=5
history= model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    epochs = EPOCHS_,
    steps_per_epoch=len(train_generator),    
    validation_steps=len(validation_generator)
#     batch_size=64,
#    callbacks=callbacks_list
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
181/181 [==============================] - 2466s 13s/step - loss: 0.4484 - accuracy: 0.8009 - val_loss: 0.3048 - val_accuracy: 0.8657
Epoch 2/5
181/181 [==============================] - 114s 628ms/step - loss: 0.3590 - accuracy: 0.8446 - val_loss: 0.6206 - val_accuracy: 0.7642
Epoch 3/5
181/181 [==============================] - 113s 625ms/step - loss: 0.3169 - accuracy: 0.8681 - val_loss: 0.3073 - val_accuracy: 0.8750
Epoch 4/5
181/181 [==============================] - 113s 623ms/step - loss: 0.3277 - accuracy: 0.8633 - val_loss: 0.4652 - val_accuracy: 0.8293
Epoch 5/5
181/181 [==============================] - 112s 620ms/step - loss: 0.3105 - accuracy: 0.8754 - val_loss: 0.3634 - val_accuracy: 0.8580


In [15]:

# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16.h5')

In [17]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
model = load_model('model_vgg16.h5')

img = image.load_img('/content/drive/MyDrive/Final-Dataset4000-SPLIT/val/Covid/2966893D-5DDF-4B68-9E2B-4979D5956C8E.jpeg',target_size = (224,224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
img_data = preprocess_input(x)
classes = model.predict(img_data)